In [119]:
import os
import shutil
import time
from threading import Timer
from time import gmtime, strftime
import numpy as np
from matplotlib import pyplot
import rasterio
from rasterio.enums import Resampling
from shapely.geometry import mapping, Polygon
from rasterio.features import shapes
import warnings
import csv
warnings.filterwarnings(action='ignore') 

In [120]:
# Global variables:
systemCooldown = 2
Error_Limit = 1
mode = True

In [121]:
# File Paths
Drive = "/Users/imdsk/Developer/Rida/RIDA/Sentinel_Process/"
Image = os.path.join(Drive, "Image")
Image_Pre = os.path.join(Drive, "Image_Pre/")
Image_Finish = os.path.join(Drive, "Image_Finish/")
Image_Missing = os.path.join(Drive, "Image_Missing/")
Output = os.path.join(Drive, "Output/")
Rtbcon = os.path.join(Drive, "Raster_BurnCon/")
Rtbreg = os.path.join(Drive, "Raster_BurnReg/")
RtbShape = os.path.join(Drive, "Raster_BurnShape/")
Track_arr = ["T4/", "T104/", "T61/", "T18/", "T118/"]

In [122]:
# Function to load cooldown
def loadCooldown():
    global mode

In [123]:
# Function to print time
def print_time():
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [124]:
# Function to move file
def Move_File(FileName, CurrDir, DestDir):
    try:
        if os.path.exists(os.path.join(CurrDir, FileName)):
            if os.path.exists(os.path.join(DestDir, FileName)):
                os.remove(os.path.join(DestDir, FileName))
            shutil.copy(os.path.join(CurrDir, FileName), DestDir)
            t = Timer(1, loadCooldown)
            t.start()
            t.join()
            os.remove(os.path.join(CurrDir, FileName))
            print(print_time() + f"Raster_Process :: Move File {FileName} Complete")
    except Exception as e:
        print(print_time() + f"Raster_Process :: Can not Move File {FileName}")
        print(print_time() + str(e))

In [125]:
def write_to_csv(data, file_name_csv):
    try:
        # Convert NumPy array to a nested Python list or dictionary
        if isinstance(data, np.ndarray):
            data_list = data.tolist()  # Convert ndarray to a nested list
        else:
            data_list = data  # For other non-ndarray data, assume it's already in list format

        # Write to CSV file
        with open(file_name_csv, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            for row in data_list:
                csv_writer.writerow(row)
        print(f"Data successfully written to {file_name_csv}")

    except Exception as e:
        print(f"Error writing to CSV file: {e}")

In [126]:
def print_with_tag(data, tag):
    print("------")
    print(tag)
    print(data)
    print("------")
    

In [127]:
# Function to process raster
def Raster_Process(Track):
    global mode, Image, Image_Pre, Image_Finish, Image_Missing, Output, Error_Limit

    Loop_Limit = 0
    image_track = os.path.join(Image, Track)
    output_track = os.path.join(Output, Track)

    rasters = [r for r in os.listdir(image_track) if r.endswith('B12.jp2')]

    for raster in rasters:
        if Loop_Limit > 0:
            mode = True
            return

        Full_name = os.path.splitext(raster)[0]
        Full_name_parts = Full_name.split('_')
        Mid_name = Full_name[:23]
        Short_name = Full_name_parts[0] + '_' + Full_name_parts[1]
        # print(Full_name)
        # print(Short_name)

        BFB03 = os.path.join(Image_Pre, Track, f"{Short_name}_B03.jp2")
        BFB04 = os.path.join(Image_Pre, Track, f"{Short_name}_B04.jp2")
        BFB08 = os.path.join(Image_Pre, Track, f"{Short_name}_B08.jp2")
        BFB12 = os.path.join(Image_Pre, Track, f"{Short_name}_B12.jp2")
        BFB1210 = os.path.join(Image_Pre, Track, f"{Short_name}_B1210.jp2")
        BFNBR = os.path.join(Image_Pre, Track, f"{Short_name}_NBR.tif")

        AFB03 = os.path.join(Image, Track, f"{Short_name}_B03.jp2")
        AFB04 = os.path.join(Image, Track, f"{Short_name}_B04.jp2")
        AFB08 = os.path.join(Image, Track, f"{Short_name}_B08.jp2")
        AFB12 = os.path.join(Image, Track, f"{Short_name}_B12.jp2")
        AFB1210 = os.path.join(Image, Track, f"{Short_name}_B1210.jp2")

        print(BFB12)
       

        if all(
            [
                os.path.exists(BFB03), os.path.exists(BFB04), os.path.exists(BFB08), os.path.exists(BFB12),
                os.path.exists(AFB03), os.path.exists(AFB04), os.path.exists(AFB08), os.path.exists(AFB12)
            ]
        ):
            print(print_time() + "Raster_Process :: Start GIS Process Please Wait....")
            Loop_Limit += 1
            # Delete Old File
            print(print_time() + "Raster_Process :: Delete Old File")
            files = os.listdir(output_track)
            for f in files:
                try:
                    os.remove(os.path.join(output_track, f))
                except Exception as e:
                    print(print_time() + f"Raster_Process :: Cannot Delete {f}")
                    print(print_time() + str(e))
                    break

            print(print_time() + "Raster_Process :: Delete Old File Complete")
            t = Timer(3, loadCooldown)
            t.start()
            t.join()

            
            def resample_raster(BFB12, Image_Pre, new_resolution):
                with rasterio.open(BFB12) as src:
                        transform = src.transform
                        data = src.read(
                            out_shape=(
                                src.count,
                                int(src.height * src.transform.a / new_resolution),
                                int(src.width * src.transform.a / new_resolution)
                            ),
                            resampling=Resampling.nearest  # You can adjust the resampling method here
                        )
                        profile = src.profile
                        profile.update(transform=transform, width=data.shape[2], height=data.shape[1])

                        with rasterio.open(Image_Pre, 'w', **profile) as dst:
                            dst.write(data)

                            
            def con(condition_array, true_array, false_array):
                return np.where(condition_array, true_array, false_array)
            

            def copy_raster(burnCon_Final, Rtbcon, new_resolution):
                with rasterio.open(burnCon_Final) as src:
                    transform = src.transform
                    data = src.read(
                        out_shape=(
                            src.count,
                            int(src.height * src.transform.a / new_resolution),
                            int(src.width * src.transform.a / new_resolution)
                        ),
                        resampling=Resampling.nearest
                    )
                    profile = src.profile
                    profile.update(transform=transform, width=data.shape[2], height=data.shape[1])

                    with rasterio.open(Rtbcon, 'w', **profile) as dst:
                        # Explicitly set the data type to 1-bit
                        profile.update(dtype='uint8', count=1, compress='packbits')
                        dst.write(data.astype('uint8'), 1)


            if os.path.exists(BFB1210) == False:
                print(print_time() + "Raster_Process :: Resample " + Short_name)

                    # Usage:
                input_raster = BFB12
                output_raster = BFB1210
                new_resolution = 10  # Change this to the desired resolution

                resample_raster(input_raster, output_raster, new_resolution)

                # ds = rasterio.open(BFB1210)
                # print(ds.read(12))
                # print("asdasd")      # (array([0, 2, 5]),)
                
                # arcpy.Resample_management(BFB12, Image_Pre + Track + Short_name + "_B1210.jp2","10 10","NEAREST")

            if os.path.exists(BFNBR) == False:
                print(print_time()+"Raster_Process :: NBR " + Short_name)
                with rasterio.open(BFB08) as src_BFB08, rasterio.open(BFB1210) as src_BFB1210:
                    data_BFB08 = src_BFB08.read(1)  # Read the data of BFB08 raster band 1
                    data_BFB1210 = src_BFB1210.read(1)  # Read the data of BFB1210 raster band 1

                    NBR_data = (data_BFB08 - data_BFB1210) / (data_BFB08 + data_BFB1210)

                    profile = src_BFB08.profile  # Use BFB08 profile as a template for the new raster
                    output_path_NBR = Image_Pre + Track + Short_name + "_NBR.tif"
                    with rasterio.open(output_path_NBR, 'w', **profile) as dst:
                        dst.write(NBR_data, 1)

                # NBR_CMD = Float(Raster(BFB08) - Raster(BFB1210)) / Float(Raster(BFB08) + Raster(BFB1210))
                # NBR_CMD.save(Image_Pre + Track + Short_name + "_NBR.tif")
                # del NBR_CMD
                        

            if os.path.exists(AFB1210) == False:
                print(print_time()+"Raster_Process :: Resample " + Mid_name)

                # Usage:
                input_raster = AFB12
                output_raster = AFB1210
                new_resolution = 10 

                resample_raster(input_raster, output_raster, new_resolution)
                # arcpy.Resample_management(AFB12, Image + Track + Mid_name + "B1210.jp2","10 10","NEAREST")

            try:
                ########################################## Raster PROCESS ################################################
                print(print_time()+"Raster_Process :: Raster Process " + Full_name[:22])

                with rasterio.open(BFNBR) as src_BFNBR, rasterio.open(AFB08) as src_AFB08, rasterio.open(AFB1210) as src_AFB1210, rasterio.open(AFB04) as src_AFB04, rasterio.open(AFB03) as src_AFB03:
                    data_BFNBR = src_BFNBR.read(1) 
                    data_AFB03 = src_AFB03.read(1)  
                    # data_AFB04 = src_AFB04.read(1)  
                    data_AFB08 = src_AFB08.read(1) 
                    data_AFB1210 = src_AFB1210.read(1)  

                    print_with_tag(data_BFNBR,'data_BFNBR')
                    print_with_tag(data_AFB03, 'data_AFB03')
                    print_with_tag(data_AFB08, 'data_AFB08')
                    print_with_tag(data_AFB1210, 'data_AFB1210')


                    PreNBR_data = (data_BFNBR)
                    print_with_tag(PreNBR_data, 'PreNBR_data')
                    PostNBR_data = (data_AFB08 - data_AFB1210) / (data_AFB08 + data_AFB1210)
                    print_with_tag(PostNBR_data, 'PostNBR_data')

                    # PNDVI_data = (data_AFB08 - data_AFB1210) / (data_AFB08 + data_AFB1210)
                    

                ########################################## Burn Condition ################################################
               
                print(print_time()+"Raster_Process :: Burn Raster Condition")

                
                dNBR = PreNBR_data - PostNBR_data
                print(dNBR)
                print("before condition")
                burnCon_LV1 = con(((dNBR > 0.27) & (PostNBR_data < 0.10)),1,0) 
                write_to_csv(burnCon_LV1, 'burncon_lv1.csv')
                print(burnCon_LV1)
                print("before Lv1")
                burnCon_Final = con(((burnCon_LV1 == 1) & (data_AFB08 < 2000) & (data_AFB08 > data_AFB03)),1,0)
                write_to_csv(burnCon_Final, 'burnCon_Final.csv')
                print(burnCon_Final)
                print("before condition final")

                
               
                
                source_raster = burnCon_Final
                destination_raster = os.path.join(Rtbcon, Track + Full_name[:15] + ".tif")
                new_resolution = 10 

                copy_raster(source_raster, destination_raster, new_resolution)
                
                # dNBR = Pre_nbr - Post_nbr
                # burnCon_LV1 = Con(((dNBR > 0.27) & (Post_nbr < 0.10)),1)
                # burnCon_Final = Con(((burnCon_LV1 == 1) & (Raster(AFB08) < 2000) & (Raster(AFB08) > Raster(AFB03))),1)
                    
                # arcpy.CopyRaster_management(burnCon_Final, Rtbcon + Track + Full_name[:15] + ".tif","DEFAULTS","","","","","1_BIT")
                
                del dNBR
                del PreNBR_data
                del PostNBR_data
                ##########################################################################################################
                
                print(print_time()+"Raster_Process :: RegionGroup")
                burnRegionGrp = RegionGroup(burnCon_Final,"FOUR","WITHIN","NO_LINK","")
                print(print_time()+"Raster_Process :: Region > 400")
                burnRegionGrp400 = Con(burnRegionGrp,1,0,"Count > 400")
                burnRegion = Con((burnRegionGrp400 == 1),1)
                print(print_time()+"Raster_Process :: Save Raster")
                

                arcpy.CopyRaster_management(burnRegion, Rtbreg + Track + Full_name[:15] + ".tif","DEFAULTS","","","","","1_BIT")
                print(print_time()+"Raster_Process :: Burn Raster Process Complate")

                ########################################## Vecter PROCESS #################################################
                burnReclass = Reclassify(burnRegion, "Value", "1 1",)
                
                arcpy.RasterToPolygon_conversion(burnReclass, RtbShape + Track + Full_name[:15], "SIMPLIFY", "VALUE")
                del burnReclass

                ########################################## Move File ######################################################
                del burnCon_LV1
                del burnCon_Final
                del burnRegionGrp
                del burnRegionGrp400
                del burnRegion
                
                print(print_time()+"Raster_Process :: Start Move File ")
                Move_File(Mid_name + "B03.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B04.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B08.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B12.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.aux.xml", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.ovr", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.xml", Image + Track, Image_Finish + Track)
                Error_Limit = 2
                print(print_time()+"Raster_Process :: Raster_Process ALL Complate >>>>>>>>>>>>>>>>>>>> \n \n")
                ###########################################################################################################

            except Exception as e:
                print(print_time()+"Raster_Process :: !!!!!!!!!! RASTER ERROR !!!!!!!!!!")
                print(print_time() + str(e))
                Error_Limit = Error_Limit - 1
                if Error_Limit < 1 :
                    print(print_time()+"Raster_Process :: !!!!!!!!!! RASTER ERROR 2 Time MoveFile to Image_Missin")
                    Move_File(Mid_name + "B03.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B04.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B08.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B12.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.aux.xml", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.ovr", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.xml", Image + Track, Image_Missing + Track)
                    Error_Limit = 2
                    
            ############################################## END Raster PROCESS################################################
        
            # ... 
        
        else:
            print(print_time() + f"Raster_Process gg111 :: {Full_name[:22]} Image not Found !!!!")
            print(BFB03, "_", os.path.exists(BFB03))
            print(BFB04, "_", os.path.exists(BFB04))
            print(BFB08, "_", os.path.exists(BFB08))
            print(BFB12, "_", os.path.exists(BFB12))
            print(AFB03, "_", os.path.exists(AFB03))
            print(AFB04, "_", os.path.exists(AFB04))
            print(AFB08, "_", os.path.exists(AFB08))
           
            print("Joined Path:", BFB03)
            Move_File(f"{Mid_name}B03.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B04.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B08.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B12.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.aux.xml", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.ovr", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.xml", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            # ... (similar moves for other files)
    print(print_time() + "Wait New Raster ::")
    mode = True

In [128]:
def main():
    current_dir = os.getcwd()

            # Print the current working directory
    print("Current Working Directory:", current_dir)
    global mode, Image_Post, Track_arr
    print(print_time() + "Application Start ::")
    while True:
        t = Timer(systemCooldown, loadCooldown)
        t.start()
        t.join()
        if mode:
            for Track in Track_arr:
                image_track = os.path.join(Image, Track)
                if os.path.exists(image_track):
                    rasters = [r for r in os.listdir(image_track) if r.endswith('B12.jp2')]
                    if rasters:
                        mode = False
                        print(print_time() + "Found NEW Raster :: ")
                        try:
                            Raster_Process(Track)
                        except Exception as e:
                            print(print_time() + "!!!!!!!!!SYSTEM ERROR !!!!!!!!!!")
                            print(print_time() + str(e))
                        print(print_time() + "Wait New Raster ::")
                else:
                    print(f"Directory not found: {image_track}")  # Inform if the directory does not exist

#Run the main function
main()


# BFB08Test = "/Users/imdsk/Developer/Rida/RIDA/Sentinel_Process/Image_Pre/T4/MSK_QUALIT_B12.jp2"
# ds = rasterio.open(BFB08Test)
# array = ds.read(1)
# print(array)
# write_to_csv(array, 'BFB08Test.csv') 
# print("--------------------")
# pyplot.imshow(array, cmap='grey')
# pyplot.show()
                  

# BFB12Test = "/Users/imdsk/Developer/Rida/RIDA/Sentinel_Process/Image_Pre/T4/MSK_QUALIT_B1210.jp2"
# ds = rasterio.open(BFB12Test)
# array = ds.read(1)
# print(array)
# # pyplot.imshow(array, cmap='grey')
# # pyplot.show() 


Current Working Directory: /Users/imdsk/Developer/Rida/RIDA
2024-01-11 19:33:08Application Start ::
2024-01-11 19:33:10Found NEW Raster :: 
/Users/imdsk/Developer/Rida/RIDA/Sentinel_Process/Image_Pre/T4/MSK_QUALIT_B12.jp2
2024-01-11 19:33:10Raster_Process :: Start GIS Process Please Wait....
2024-01-11 19:33:10Raster_Process :: Delete Old File
2024-01-11 19:33:10Raster_Process :: Delete Old File Complete
2024-01-11 19:33:13Raster_Process :: Resample MSK_QUALIT
2024-01-11 19:33:46Raster_Process :: NBR MSK_QUALIT
2024-01-11 19:33:55Raster_Process :: Resample MSK_QUALIT_B12
2024-01-11 19:34:21Raster_Process :: Raster Process MSK_QUALIT_B12
------
data_BFNBR
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
------
------
data_AFB03
[[1922 2054 2100 ... 2418 2406 2412]
 [2010 2108 2118 ... 2422 2406 2410]
 [2058 2092 2012 ... 2422 2416 2440]
 ...
 [1644 1724 2306 ... 1876 1930 1898]
 [2094 2558 2830 ... 1865 1874 1882]
 [

KeyboardInterrupt: 